In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel, AutoModel
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch
from torch.optim import AdamW
# 
from dataset import EssayDataset
from longDataset import LongEssayDataset
from bert_regression import BertRegressionModel
from hierarchicalBert import HierarchicalBert

In [2]:
df = pd.read_csv("dataset/aes_dataset.csv")
df.head()

,question,reference_answer,answer,score,dataset,max_length1,normalized_score
0,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sumber tenaga, pemanis alami, menjaga sistem i...",27.0,analisis_essay,65,0.27
1,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"sebagai sumber energi, pemanis alami, menjaga ...",21.0,analisis_essay,66,0.21
2,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,1. Sebagai energi. 2. Sebagai memperlancaar pe...,42.0,analisis_essay,76,0.42
3,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,"untuk membuat kenyang, agar tidak lapar, agar ...",18.0,analisis_essay,67,0.18
4,Jelaskan kegunaan karbohidrat untuk tubuh kita.,Fungsi karbohidrat adalah sebagai pemasok ener...,Karbohidrat mempunyai peran penting untuk pros...,82.0,analisis_essay,105,0.82


# Training

In [3]:
df_sag = df[df['dataset'] == 'sag']
df_sag.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2558 entries, 19376 to 21933
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          2558 non-null   object 
 1   reference_answer  2558 non-null   object 
 2   answer            2558 non-null   object 
 3   score             2558 non-null   float64
 4   dataset           2558 non-null   object 
 5   max_length1       2558 non-null   int64  
 6   normalized_score  2558 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 159.9+ KB


In [4]:
df_test = df_sag[:12]
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12 entries, 19376 to 19387
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   question          12 non-null     object 
 1   reference_answer  12 non-null     object 
 2   answer            12 non-null     object 
 3   score             12 non-null     float64
 4   dataset           12 non-null     object 
 5   max_length1       12 non-null     int64  
 6   normalized_score  12 non-null     float64
dtypes: float64(2), int64(1), object(4)
memory usage: 768.0+ bytes


In [5]:
test = df[df['max_length1'] > 1000]

In [6]:
# load indobert tokenizer
tokenizer1 = BertTokenizer.from_pretrained("indobenchmark/indobert-lite-base-p2")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [7]:
# apply dataset with overlapping 128 and max length 512
test_dataset = LongEssayDataset(test, tokenizer1, 512, 128)

In [9]:
# load dataset to dataloader for creating batch size 4
dataloader = DataLoader(test_dataset, batch_size=1, collate_fn=lambda x: list(zip(*x)))

In [10]:
# load model, optimizer, and loss function (MSE)
model = HierarchicalBert("indobenchmark/indobert-lite-base-p2")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.MSELoss()
model.train()

HierarchicalBert(
  (bert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(30000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertSdpaAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
 

In [11]:
# training process
for epoch in range(1):
    for batch, targets in dataloader:
        optimizer.zero_grad()
        # batch = batch.to(device)
        targets = torch.stack(targets).to(device)
        # forward pass
        predictions = model(batch).squeeze(1)
        print("Predictions:", predictions)
        print("Targets:", targets)
        # compute loss
        loss = criterion(predictions, targets)
        print("loss : ", loss.item())
        # backward pass
        loss.backward()
        optimizer.step()

c:\Users\User\Documents\Code\env\lib\site-packages\transformers\models\albert\modeling_albert.py:404: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attention_output = torch.nn.functional.scaled_dot_product_attention(


stacked chunks  tensor([[ 0.2031, -0.1543,  0.7005,  ..., -0.3017,  0.8910,  0.2358],
        [ 0.0139, -0.0067,  0.1115,  ...,  0.0378, -0.0385,  0.1059],
        [ 0.0197, -0.0284,  0.1244,  ...,  0.0484, -0.0558,  0.0994]],
       device='cuda:0', grad_fn=<CatBackward0>)
stacked chunks transponse tensor([[ 0.2031,  0.0139,  0.0197],
        [-0.1543, -0.0067, -0.0284],
        [ 0.7005,  0.1115,  0.1244],
        ...,
        [-0.3017,  0.0378,  0.0484],
        [ 0.8910, -0.0385, -0.0558],
        [ 0.2358,  0.1059,  0.0994]], device='cuda:0',
       grad_fn=<PermuteBackward0>)
attention weight  tensor([[1.0000, 0.0000, 0.0000],
        [0.0000, 0.7090, 0.2910],
        [0.0000, 0.6807, 0.3193]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
pooled output  tensor([[ 0.2031, -0.1543,  0.7005,  ..., -0.3017,  0.8910,  0.2358],
        [ 0.0156, -0.0130,  0.1152,  ...,  0.0409, -0.0435,  0.1040],
        [ 0.0157, -0.0136,  0.1156,  ...,  0.0412, -0.0440,  0.1038]],
       device='cuda